In [4]:
import pandas as pd
import nltk
import numpy as np
from collections import defaultdict
from nltk import word_tokenize
from pymystem3 import Mystem
from nltk.corpus import stopwords
stop = set(stopwords.words())
import gensim.models as gen

# сконвертируем эмотиконы в текст

import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

from gensim.models.wrappers import FastText 

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, precision_recall_curve, roc_curve, roc_auc_score
import lightgbm as lgb
import multiprocess

In [5]:
!head '../../data/sentiment_analysis/train.csv'

ItemID,Sentiment,SentimentText
1,0,                     is so sad for my APL friend.............
2,0,                   I missed the New Moon trailer...
3,1,              omg its already 7:30 :O
4,0,          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...
5,0,         i think mi bf is cheating on me!!!       T_T
6,0,         or i just worry too much?        
7,1,       Juuuuuuuuuuuuuuuuussssst Chillin!!
8,0,       Sunny Again        Work Tomorrow  :-|       TV Tonight
9,1,      handed in my uniform today . i miss you already


In [142]:
train = pd.read_csv('../../data/sentiment_analysis/train.csv', sep=',', encoding='ISO-8859-1')
test = pd.read_csv('../../data/sentiment_analysis/test.csv', sep=',', encoding='ISO-8859-1')

In [7]:
train.shape

(99989, 3)

In [8]:
test.shape

(299989, 2)

In [9]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [10]:
train.Sentiment.value_counts()

1    56457
0    43532
Name: Sentiment, dtype: int64

In [11]:
#    классы- сбалансированны

In [12]:
data_tr = train['SentimentText']
data_ts = test['SentimentText']

In [13]:
data_full = pd.concat([data_tr, data_ts], axis=0)
data_full = data_full.values

In [14]:
data_full = ' '.join([x for x in list(data_full)])

In [15]:
data_full[:200] 

"                     is so sad for my APL friend.............                    I missed the New Moon trailer...               omg its already 7:30 :O           .. Omgaga. Im sooo  im gunna CRy. I've"

In [16]:
m = Mystem()
def lemmatize_word(word):
    try:
        return m.lemmatize(word)[0] 
    except:
        return ''

In [17]:
!pip install emot

You should consider upgrading via the 'pip install --upgrade pip' command.


#### Заменим эмоджи и эмотиконы на текст  

In [18]:
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

# Function for converting emoticons into word
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

# Example
text1 = "Hilarious 😂. The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒"
convert_emojis(text1)

'Hilarious face_with_tears_of_joy. The feeling of making a sale smiling_face_with_sunglasses, The feeling of actually fulfilling orders unamused_face'

In [19]:
text = "Hello :-) :-)"
convert_emoticons(text)

'Hello Happy_face_smiley Happy_face_smiley'

In [20]:
data_full = convert_emoticons(convert_emojis(data_full))

In [21]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' URL ', text)
    text = re.sub('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', ' email ', text)
    text = re.sub('(?:\\+7|8)[ -]?\\(?9\\d{2}\\)?[ -]?\\d{3}[ -]?\\d{2}[ -]?\\d{2}', ' phone ', text)
    text = re.sub('@[^\s]+', ' USER ', text)
    text = re.sub(' +', ' ', text)
   
    text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text)
    text = re.sub(' +', ' ', text)
    text = convert_emoticons(convert_emojis(text))
    return (text.strip())

In [22]:
def get_metrics(labels, predictions):
    rep = classification_report(labels, predictions, output_dict=True)
    return round(rep['macro avg']['f1-score'],3), round(rep['weighted avg']['f1-score'],3),\
           round(rep['macro avg']['recall'],3), round(rep['weighted avg']['recall'],3),\
           round(rep['micro avg']['recall'],3)

In [31]:
def preproc_text(text, w2w):
    return ' '.join([w2w[word] for word in text.split() if (len(word)) and w2w[word] not in stop])

In [24]:
def text2vec(text, ft_emb):
    vec = np.array(list(map(lambda x: ft_emb[x], text.split())))
    return np.append(vec.shape[0], np.append(np.append(np.append(vec.mean(axis=0), vec.max(axis=0)), vec.min(axis=0)), vec.std(axis=0)))

In [25]:
train = train[~(train['SentimentText'].isnull())& \
              ~(train['Sentiment'].isnull())   
             ]

In [26]:
res = list(map(lambda x: preprocess_text(x), train['SentimentText']))
train['description_preproc'] = list(map(lambda x: x, res))
del res

In [27]:
corpus = set()
a = list(map(lambda x: corpus.update(x.split()), train['description_preproc']))
del a
w2w = defaultdict()
for word in corpus:
    w2w[word] = m.lemmatize(word)[0]

In [28]:
train['description_preproc'] = list(map(lambda x: preproc_text(x, w2w), train['description_preproc']))
train = train[train['description_preproc'] != '']

In [26]:
ft_emb = gen.FastText.load_fasttext_format('../../data/wiki.en.bin')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [27]:
X = np.array(list(map(lambda x: text2vec(x, ft_emb),\
                     train['description_preproc']
                     )
                 )  
            )                  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [28]:
y = train['Sentiment'].values

In [29]:
# попробуем логистическую регрессию как классификатор

In [37]:
kf = StratifiedKFold(n_splits=5, random_state=1234567890, shuffle=True)
lr_scores = pd.DataFrame()
for dev_idx, val_idx in kf.split(X, y):
    break
X_train = X[dev_idx]
y_train = y[dev_idx]
X_val = X[val_idx]
y_val = y[val_idx]


model = LogisticRegression(C=0.025, max_iter=100, tol=0.05, multi_class='auto', class_weight='balanced',
                           random_state=13)
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)
#print ('train: avg_f1 = {}, weighted_f1 = {}, macro_rec {}, weight_rec = {}, micro_rec = {}'.format(*get_metrics(y_train, pred_train)))
#print ('val: avg_f1 = {}, weighted_f1 = {}, macro_rec {}, weight_rec = {}, micro_rec = {}'.format(*get_metrics(y_val, pred_val)))
print(classification_report(y_train, pred_train))
print(classification_report(y_val, pred_val))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.67      0.73      0.69     34804
           1       0.77      0.72      0.75     45159

    accuracy                           0.72     79963
   macro avg       0.72      0.72      0.72     79963
weighted avg       0.73      0.72      0.72     79963

              precision    recall  f1-score   support

           0       0.66      0.72      0.69      8701
           1       0.77      0.71      0.74     11290

    accuracy                           0.72     19991
   macro avg       0.71      0.72      0.71     19991
weighted avg       0.72      0.72      0.72     19991



In [35]:
# используем веса от логистической регрессии как фичи для градиентного бустинга

In [38]:
pred_train = model.predict_proba(X_train)
pred_val = model.predict_proba(X_val)
print(np.hstack((pred_train, X_train)).shape)
print(np.hstack((pred_val, X_val)).shape)

(79963, 1203)
(19991, 1203)


In [39]:
%%time
clf = lgb.LGBMClassifier(n_estimators=1000, #800
                         max_depth=5, #3
                         learning_rate=0.03, #0.03
                         metric='logloss', #'multi_logloss'
                         objective='binary', #'multiclass'
                         #num_class=2,
                         reg_lambda=1,
                         class_weight='balanced',
                         colsample_bytree = 0.5,
                         subsample= 0.5,
                         n_jobs=multiprocess.cpu_count()-1)
clf.fit(X_train, y_train)
pred_train = clf.predict(X_train)
pred_val = clf.predict(X_val)
#print ('train: avg_f1 = {}, weighted_f1 = {}, macro_rec {}, weight_rec = {}, micro_rec = {}'.format(*get_metrics(y_train, pred_train)))
#print ('val: avg_f1 = {}, weighted_f1 = {}, macro_rec {}, weight_rec = {}, micro_rec = {}'.format(*get_metrics(y_val, pred_val)))
print(classification_report(y_train, pred_train))
print(classification_report(y_val, pred_val))

              precision    recall  f1-score   support

           0       0.80      0.85      0.83     34804
           1       0.88      0.84      0.86     45159

    accuracy                           0.84     79963
   macro avg       0.84      0.84      0.84     79963
weighted avg       0.85      0.84      0.84     79963

              precision    recall  f1-score   support

           0       0.69      0.72      0.70      8701
           1       0.78      0.75      0.76     11290

    accuracy                           0.74     19991
   macro avg       0.73      0.73      0.73     19991
weighted avg       0.74      0.74      0.74     19991

CPU times: user 6h 23min 49s, sys: 2min 54s, total: 6h 26min 43s
Wall time: 14min 2s


#### Градиентный бустинг дал только 0.74 f1 на тесте
#### попробуем RNN

In [40]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import time
import os
from torch.autograd import Variable
import numpy as np
from sklearn.model_selection import train_test_split

In [143]:
res = list(map(lambda x: preprocess_text(x), train['SentimentText']))
train['description_preproc'] = list(map(lambda x: x, res))
del res

In [144]:
X = train['SentimentText']
y = train['Sentiment']

In [145]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [146]:
text = ' '.join(train['description_preproc'].to_list())

In [147]:
ALL_WORDS = set(text.split(' '))

In [148]:
from collections import Counter

In [149]:
c = Counter(text.split(' '))
c.most_common(50)

[('USER', 88978),
 ('i', 63432),
 ('you', 30105),
 ('the', 30014),
 ('to', 29616),
 ('a', 22398),
 ('it', 22330),
 ('and', 16449),
 ('my', 13873),
 ('that', 13719),
 ('for', 12573),
 ('s', 12486),
 ('t', 12277),
 ('is', 12118),
 ('in', 11950),
 ('me', 11196),
 ('of', 10626),
 ('have', 9986),
 ('on', 9677),
 ('so', 9229),
 ('but', 9126),
 ('quot', 9122),
 ('m', 8894),
 ('be', 7464),
 ('was', 7250),
 ('not', 7247),
 ('just', 6865),
 ('your', 6480),
 ('can', 6119),
 ('are', 6100),
 ('good', 5927),
 ('with', 5552),
 ('like', 5538),
 ('no', 5523),
 ('lol', 5399),
 ('at', 5342),
 ('get', 5324),
 ('u', 5284),
 ('we', 5278),
 ('too', 5261),
 ('all', 4846),
 ('up', 4808),
 ('now', 4799),
 ('this', 4761),
 ('do', 4670),
 ('know', 4611),
 ('love', 4517),
 ('what', 4399),
 ('out', 4213),
 ('thanks', 4117)]

In [150]:
len(c)

50160

In [151]:
WORDS_COUNT = 10000
ALL_WORDS = set([w for w, _ in c.most_common(WORDS_COUNT)])
INDEX_TO_WORD = ['<pad>', '<miss>'] + list(ALL_WORDS)
WORD_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_WORD)}
MAX_LEN = 150

In [152]:
train_data = torch.zeros((len(X_train), MAX_LEN), dtype=int)

In [153]:
test_data = torch.zeros((len(X_val), MAX_LEN), dtype = int)

In [154]:
X_train = X_train.to_list()

In [155]:
X_val = X_val.to_list()

In [156]:
%%time
for i in range(len(X_train)):
    for j, w in enumerate(X_train[i].split(' ')):
        if j >= MAX_LEN:
            break
        train_data[i, j] = WORD_TO_INDEX.get(w, WORD_TO_INDEX['<miss>'])

CPU times: user 10.6 s, sys: 36 ms, total: 10.6 s
Wall time: 10.6 s


In [157]:
%%time
for i in range(len(X_val)):
    for j, w in enumerate(X_val[i].split(' ')):
        if j >= MAX_LEN:
            break
        test_data[i, j] = WORD_TO_INDEX.get(w, WORD_TO_INDEX['<miss>'])

CPU times: user 2.64 s, sys: 4 ms, total: 2.64 s
Wall time: 2.64 s


In [158]:
y_train = y_train.to_list()
y_val = y_val.to_list()

In [159]:
train_dataset = torch.utils.data.TensorDataset(train_data, torch.LongTensor(y_train))
test_dataset = torch.utils.data.TensorDataset(test_data, torch.LongTensor(y_val))

In [160]:
BATCH_SIZE = 256

In [161]:
train = torch.utils.data.DataLoader(train_dataset, BATCH_SIZE, shuffle=False)
test = torch.utils.data.DataLoader(test_dataset, BATCH_SIZE, shuffle=False)

In [162]:
dev = torch.device('cuda')

In [163]:
class SimpleGRU(nn.Module):
    def __init__(self, dict_size, embed_size, num_hiddens, num_layers=2, dropout=0.3):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embed = nn.Embedding(dict_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, batch_first=True)
        
    def forward(self, X):
        output = self.embed(X)
        y, s = self.rnn(output)
        return y, s

In [164]:
class RnnModel(nn.Module):
    def __init__(self, dict_size, embed_size, num_hiddens, num_classes):
        super().__init__()

        self.SimpleRNN = SimpleGRU(dict_size, embed_size, num_hiddens)
        
        self.classifier1 = nn.Sequential(
            nn.BatchNorm1d(num_hiddens),
            nn.Linear(num_hiddens, num_hiddens*2)
        )
        self.classifier2 = nn.Sequential(    
            nn.BatchNorm1d(num_hiddens*2),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(num_hiddens*2, num_classes)
        )
        
        
    def forward(self, X1):
        y1, s1 = self.SimpleRNN(X1)
                
        out = self.classifier2(self.classifier1(s1[0]))
        
        return out

In [ ]:
def validate_acc(model, dataset1, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    loss_acc, passed, correct = 0., 0, 0
    for A1 in dataset1:
        X1 = A1[0].to(dev)
        y1 = A1[1].to(dev)
            
    
        output = model(X1)
        l = loss(output, y1)

        loss_acc += l.item()
        correct += (output.argmax(dim=1) == y1).sum().item()
        passed += len(y1)
        
      

    return loss_acc / passed, correct / passed

In [166]:
epochs = 20
torch.manual_seed(7)

In [122]:
loss_weights = torch.FloatTensor([1./16., 1.-1./16.]).to(dev)

In [123]:
def train_model(model, train_dl1, test_dl1, trainer, dev, num_epochs=epochs, ckpt_path = os.path.join('../../data/models/', 'Simple_GRU'+'.pt')):
    best_record = 0.0
    loss = nn.CrossEntropyLoss(weight=loss_weights, reduction='sum')
    for ep in range(num_epochs):
        ep_start, tloss_acc, tpassed, tcorrect = time.time(), 0., 0, 0
        for A1 in train_dl1:
        
            X1 = A1[0].to(dev)
            y1 = A1[1].to(dev)
            
            
            trainer.zero_grad()

            output = model(X1)
            l = loss(output, y1)
            l.backward()
            trainer.step()

            tloss_acc += l.item()
            tcorrect += (output.argmax(dim=1) == y1).sum().item()
            tpassed += len(y1)
    
        test_loss, test_acc = validate_acc(model, test_dl1, dev)
        
        state_dict = {
            'epoch': ep,
            'siamese': model.state_dict(),
            'optimizer': trainer.state_dict(),
        }
        torch.save(state_dict, ckpt_path)
        print ('Model '+ckpt_path+' saved!\n')

        print('Epoch {}. Taked {:.3f} sec. Train loss: {:.3f}, Train acc {:.3f}, Test  Loss {:.3f}, Test Acc {:.3f}'.format(
            ep, time.time() - ep_start, tloss_acc / tpassed, tcorrect / tpassed, test_loss, test_acc
        ))

In [175]:
model = RnnModel(len(INDEX_TO_WORD), 100, 100, 2).to(dev)

In [176]:
for A1 in test:
        
        
        X1 = A1[0].to(dev)
        y1 = A1[1].to(dev)

        
        labels.append(y1.data.cpu())
        output = model(X1)
        break

In [177]:
X1

tensor([[   1,    1,    1,  ...,    0,    0,    0],
        [   1,    1, 2661,  ...,    0,    0,    0],
        [   1,    1,    1,  ...,    0,    0,    0],
        ...,
        [   1, 6446, 2912,  ...,    0,    0,    0],
        [   1,  137, 2838,  ...,    0,    0,    0],
        [   1,    1, 8043,  ...,    0,    0,    0]], device='cuda:0')

In [178]:
X1.size()

torch.Size([256, 150])

In [179]:
y1.size()

torch.Size([256])

In [174]:
y1

tensor([1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
        1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0')

In [182]:
trainer = torch.optim.AdamW(model.parameters(), lr=0.01)
train_model(model, train, test, trainer, dev, num_epochs=10)

Model ../../data/models/Simple_GRU.pt saved!

Epoch 0. Taked 6.322 sec. Train loss: 0.104, Train acc 0.566, Test  Loss 1.121, Test Acc 0.563
Model ../../data/models/Simple_GRU.pt saved!

Epoch 1. Taked 6.213 sec. Train loss: 0.091, Train acc 0.577, Test  Loss 1.137, Test Acc 0.578
Model ../../data/models/Simple_GRU.pt saved!

Epoch 2. Taked 5.915 sec. Train loss: 0.086, Train acc 0.600, Test  Loss 1.104, Test Acc 0.597
Model ../../data/models/Simple_GRU.pt saved!

Epoch 3. Taked 5.528 sec. Train loss: 0.082, Train acc 0.618, Test  Loss 1.046, Test Acc 0.606
Model ../../data/models/Simple_GRU.pt saved!

Epoch 4. Taked 6.719 sec. Train loss: 0.080, Train acc 0.636, Test  Loss 1.073, Test Acc 0.610
Model ../../data/models/Simple_GRU.pt saved!

Epoch 5. Taked 4.848 sec. Train loss: 0.078, Train acc 0.652, Test  Loss 1.068, Test Acc 0.617
Model ../../data/models/Simple_GRU.pt saved!

Epoch 6. Taked 4.924 sec. Train loss: 0.078, Train acc 0.657, Test  Loss 1.076, Test Acc 0.626
Model ../../d

In [183]:
trainer = torch.optim.AdamW(model.parameters(), lr=0.001)
train_model(model, train, test, trainer, dev, num_epochs=10)

Model ../../data/models/Simple_GRU.pt saved!

Epoch 0. Taked 6.627 sec. Train loss: 0.072, Train acc 0.683, Test  Loss 1.069, Test Acc 0.647
Model ../../data/models/Simple_GRU.pt saved!

Epoch 1. Taked 6.259 sec. Train loss: 0.067, Train acc 0.715, Test  Loss 1.081, Test Acc 0.660
Model ../../data/models/Simple_GRU.pt saved!

Epoch 2. Taked 6.235 sec. Train loss: 0.062, Train acc 0.740, Test  Loss 1.109, Test Acc 0.664
Model ../../data/models/Simple_GRU.pt saved!

Epoch 3. Taked 5.692 sec. Train loss: 0.059, Train acc 0.759, Test  Loss 1.139, Test Acc 0.672
Model ../../data/models/Simple_GRU.pt saved!

Epoch 4. Taked 6.537 sec. Train loss: 0.056, Train acc 0.771, Test  Loss 1.174, Test Acc 0.673
Model ../../data/models/Simple_GRU.pt saved!

Epoch 5. Taked 6.820 sec. Train loss: 0.054, Train acc 0.785, Test  Loss 1.219, Test Acc 0.679
Model ../../data/models/Simple_GRU.pt saved!

Epoch 6. Taked 6.385 sec. Train loss: 0.052, Train acc 0.795, Test  Loss 1.267, Test Acc 0.681
Model ../../d

In [184]:
trainer = torch.optim.AdamW(model.parameters(), lr=0.0001)
train_model(model, train, test, trainer, dev, num_epochs=20)

Model ../../data/models/Simple_GRU.pt saved!

Epoch 0. Taked 6.723 sec. Train loss: 0.045, Train acc 0.824, Test  Loss 1.420, Test Acc 0.686
Model ../../data/models/Simple_GRU.pt saved!

Epoch 1. Taked 6.035 sec. Train loss: 0.044, Train acc 0.827, Test  Loss 1.431, Test Acc 0.686
Model ../../data/models/Simple_GRU.pt saved!

Epoch 2. Taked 6.775 sec. Train loss: 0.044, Train acc 0.830, Test  Loss 1.449, Test Acc 0.686
Model ../../data/models/Simple_GRU.pt saved!

Epoch 3. Taked 6.301 sec. Train loss: 0.043, Train acc 0.833, Test  Loss 1.475, Test Acc 0.687
Model ../../data/models/Simple_GRU.pt saved!

Epoch 4. Taked 6.724 sec. Train loss: 0.043, Train acc 0.834, Test  Loss 1.499, Test Acc 0.688
Model ../../data/models/Simple_GRU.pt saved!

Epoch 5. Taked 6.527 sec. Train loss: 0.042, Train acc 0.836, Test  Loss 1.519, Test Acc 0.687
Model ../../data/models/Simple_GRU.pt saved!

Epoch 6. Taked 6.179 sec. Train loss: 0.042, Train acc 0.838, Test  Loss 1.534, Test Acc 0.689
Model ../../d

In [185]:
#Inference

result = []
labels = []

for A1 in test:
        
        
        X1 = A1[0].to(dev)
        y1 = A1[1].to(dev)

        
        labels.append(y1.data.cpu())
        output = model(X1)
        output = output.to(dev)
        output = output.squeeze(0)
        sm = nn.Softmax(dim=1).to(dev)
        res = sm(output.data)[:,1]
        result += res.data.cpu().tolist() 

In [186]:
res = []
for i in result:
    if i>=.5:
        res.append(1)
    else:
        res.append(0)

In [187]:
res1 = []
for i in result:
    res1.append(i)

In [188]:
labs = np.array([])
for i in labels:
    labs = np.hstack([labs, i.numpy()])

In [191]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
print("Accuracy: ", accuracy_score(labs, res))
print("ROC_AUC: ", roc_auc_score(labs, res1))
print(classification_report(labs, res))

Accuracy:  0.6916191619161917
ROC_AUC:  0.7453181924405609
              precision    recall  f1-score   support

         0.0       0.73      0.46      0.57      8750
         1.0       0.68      0.87      0.76     11248

    accuracy                           0.69     19998
   macro avg       0.70      0.67      0.66     19998
weighted avg       0.70      0.69      0.68     19998



In [192]:
#### При помощи RNN удалось выучить классификатор до 0.68 F1, хуже чем GradientBoosting